In [ ]:
#|hide
#|default_exp serve

# serve

> fastdaemon server

In [ ]:
#|export
import importlib
import sys
from contextlib import contextmanager
from functools import partial
from io import StringIO
from multiprocessing import get_context
from socketserver import TCPServer, StreamRequestHandler

from fastcore.parallel import ProcessPoolExecutor
from fastcore.script import *

from fastdaemon.core import *

In [ ]:
#|hide
from nbprocess.showdoc import *

In [ ]:
import time
from fastcore.parallel import threaded
from pathlib import Path
from socketserver import UnixStreamServer

In [ ]:
#|export
def _setattrs(o, d):
    for k,v in d.items(): setattr(o,k,v)

In [ ]:
#|export
@contextmanager
def _redirect_streams(argv, stdin, stdout, stderr):
    new = {k:v for k,v in locals().items()}
    old = {o:getattr(sys,o) for o in new.keys()}
    _setattrs(sys, new)
    try: yield new['stdout'],new['stderr']
    finally: _setattrs(sys, old)

In [ ]:
#|export
def _run(cmd, argv, stdin, stdout, stderr):
    with _redirect_streams(argv,stdin,stdout,stderr): cmd()
    return stdout,stderr

In [ ]:
#|export
class CmdHandler(StreamRequestHandler):
    "Run `self.server.cmd` with request's `argv` and `stdin`; return `stdout` and `stderr`"
    def setup(self):
        super().setup()
        stdin,argv = recv_record(self.rfile.read)
        self.argv = [self.server.cmd.__name__] + argv.split(' ') if argv else []
        self.stdin,self.stdout,self.stderr = StringIO(stdin),StringIO(),StringIO()
        print(f'stdin={self.stdin.getvalue()} argv={self.argv}')

    def finish(self):
        self.stdout,self.stderr = self.stdout.getvalue(),self.stderr.getvalue()
        print(f'stdout={self.stdout} stderr={self.stderr}')
        send_record(self.wfile.write, (self.stdout,self.stderr))
        super().finish()

    def _handle(self, f): return self.server.pool.submit(f).result() if hasattr(self.server,'pool') else f()
    def handle(self):
        f = partial(_run, self.server.cmd, self.argv, self.stdin, self.stdout, self.stderr)
        self.stdout,self.stderr = self._handle(f)

In [ ]:
# TODO: can we not just use ForkingMixin?

`DaemonHandler`'s primary use-case is in `DaemonServer`, however it supports any `socketserver.BaseServer` that has `pool` and `cmd` attributes.

In [ ]:
def _cmd():
    sys.stdout.write(sys.stdin.read()+sys.argv[1])
    sys.stderr.write('Error!')

In [ ]:
class _CmdServer(UnixStreamServer): cmd = lambda x: _cmd()

In [ ]:
p = Path('fdaemon.sock')
if p.exists(): p.unlink()

@threaded
def _f():
    with _CmdServer(str(p), CmdHandler) as srv: srv.handle_request()
_f()
time.sleep(0.2) # wait for server to start

In [ ]:
transfer(['Hello, ', 'world!'], str(p))

stdin=Hello,  argv=['<lambda>', 'world!']
stdout=Hello, world! stderr=Error!


['Hello, world!', 'Error!']

In [ ]:
#|export
class PoolingMixin:
    "Socket server with a `cmd` and `ProcessPoolExecutor`"
    def __init__(self, server_address, cmd, RequestHandlerClass=CmdHandler, timeout=None, **kwargs):
        self.cmd = cmd
        if timeout is not None: self.timeout = timeout
        super().__init__(server_address, RequestHandlerClass)

    def server_activate(self):
        self.pool = ProcessPoolExecutor(mp_context=get_context('fork')) # TODO: make ctx configurable?
        super().server_activate()
        
    def server_close(self):
        if hasattr(self,'pool'): self.pool.shutdown()
        super().server_close()
        
    def handle_timeout(self): return True

In [ ]:
class _PoolingServer(PoolingMixin, UnixStreamServer): pass

Here's an example of how to use `DaemonServer`. First, define the `cmd`. It should have no arguments itself but rather parse its arguments from `sys.argv`. Its return value isn't used, instead it should write to `stdout`.

Then start the server. We start it with `handle_request` in a separate thread:

In [ ]:
p = Path('fdaemon.sock')
if p.exists(): p.unlink()

@threaded
def _f():
    with _PoolingServer(str(p), _cmd) as srv: srv.handle_request()
_f()
time.sleep(0.2) # wait for server to start

...so that we can send a request and print its response:

In [ ]:
transfer(['Hello, ', 'world!'], str(p))

stdin=Hello,  argv=['_cmd', 'world!']
stdout=Hello, world! stderr=Error!


['Hello, world!', 'Error!']

In [ ]:
#|export
class PoolingTCPServer(PoolingMixin, TCPServer):
    allow_reuse_address = True

In [ ]:
#|export
def _fastdaemon_serve(cmd, port, host=None, timeout=None):
    "Serve `cmd` on `port`, with optional `host` and `timeout`"
    host = host or 'localhost'
    with PoolingTCPServer((host,port), cmd, timeout=timeout) as srv:
        while not srv.handle_request(): pass

A convenient wrapper to instantiate and start a `DaemonServer` that handles requests until it's interrupted or times out. Here's the previous example using `fastdaemon_serve`. We've also set a `timeout` to avoid running forever:

In [ ]:
host,port = 'localhost',9999

@threaded
def _f(): _fastdaemon_serve(_cmd, port, host, timeout=1)
_f()
time.sleep(0.4) # wait for server to start

In [ ]:
transfer(['Hello, ', 'world!'], port, host)

stdin=Hello,  argv=['_cmd', 'world!']
stdout=Hello, world! stderr=Error!


['Hello, world!', 'Error!']

In [ ]:
#|export
def _import_cmd(cmd):
    mn, on = cmd.split(':')
    m = importlib.import_module(mn)
    return getattr(m,on)

In [ ]:
_import_cmd('nbprocess.clean:nbprocess_clean')

<function nbprocess.clean.nbprocess_clean(fname: str = None, clear_all: bool = False, disp: bool = False, stdin: bool = False)>

In [ ]:
#|export
@call_parse
def fastdaemon_serve(
    cmd:str, # Module path to callable command (example: pkg.mod:obj)
    port:int, # Server port
    host:str='localhost', # Server host
    timeout:int=None): # Shutdown after `timeout` seconds without requests
    "Serve `cmd` on `port`, with optional `host` and `timeout`"
    _cmd = _import_cmd(cmd)
    with PoolingTCPServer((host,port), _cmd, timeout=timeout) as srv:
        while not srv.handle_request(): pass

In [ ]:
!fastdaemon_serve -h

usage: fastdaemon_serve [-h] [--host HOST] [--timeout TIMEOUT] cmd port

Serve `cmd` on `port`, with optional `host` and `timeout`

positional arguments:
  cmd                Module path to callable command (example: pkg.mod:obj)
  port               Server port

optional arguments:
  -h, --help         show this help message and exit
  --host HOST        Server host (default: localhost)
  --timeout TIMEOUT  Shutdown after `timeout` seconds without requests


## Export -

In [ ]:
#|hide
#|eval: false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()